<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/kurs-analiza-danych-2022/blob/main/Tydzie%C5%84%203/Cz%C4%99%C5%9B%C4%87%C2%A03_Regresja_regularyzowana_PG_zmiany.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Regularyzacja regresji

Regresja liniowa jest jedna z najbardziej podstawowych technik uczenia maszynowego. Co jednak jeśli 
pomimo bardzo prostego modelu model okazuje się mieć cechy przeuczenia (overfit). 
Dostępne są wtedy dla nas techniki nazywane ogólnikowo regularyzacją. Określają one zasady pomniejszania
modelu, dzięki któremu zachowuje on pewne dodatkowe warunki a w efekcie pomniejsza się BIAS modelu i
przestrzeń na jego przeuczenie.

Omówimy w tej części 3 techniki regularyzacyjne

1. Regresja grzebietowa
1. Metoda Lasso
1. Elastyczna siatka

Najpierw zanim omówimy technikę, dokonajmy importu i 
preprocesssingu według przepisu z poprzednich materiałów.

In [1]:
from sklearn import datasets

dataset3 = datasets.load_diabetes()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

size = len(dataset3.data)

data3 = np.concatenate([dataset3.data, np.reshape(dataset3.target, (size,1))], axis = 1)
column_names = dataset3.feature_names[:]
column_names.append('target')
df3 = pd.DataFrame(data3, columns = column_names, copy = True)
df3.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


## Uzupełnienie

Widać, że kolumny ze zmiennymi objaśniającymi zostały standaryzowane na etapie przygotowania zbioru danych. Trudno więc będzie dodać nowe dane. Natomiast kolumna *target* nie została poddana standardyzacji. To z kolei może powodować problem z doborem parametrów przy omawianych technikach — oznacza duże wartości współczynników regresji. Dokonamy zatem jej przekształcenia.

**Zrezygnujemy za to na tym etapie z podziału na część uczącą i testową — w wypadku regresji parametrycznej nie jest ona konieczna,** zwłaszcza gdy parametrów jest o rząd wielkości mniej niż danych.

In [2]:
from sklearn.preprocessing import StandardScaler

target_data = np.reshape(dataset3.target, (size,1))
target_scaler = StandardScaler()
target_scaler.fit(target_data)
print("Średnia:", target_scaler.mean_, "Odchylenie:", target_scaler.scale_)
# tak samo skaluje się nowe dane
y2 = target_scaler.transform(target_data)
print(y2[0:5])
# można też odwrócić skalowanie
print(target_scaler.inverse_transform(y2[0:5]))

X = df3.drop('target', axis = 1)
y = df3['target']
X.head()

Średnia: [152.13348416] Odchylenie: [77.00574587]
[[-0.01471948]
 [-1.00165882]
 [-0.14457991]
 [ 0.69951294]
 [-0.22249618]]
[[151.]
 [ 75.]
 [141.]
 [206.]
 [135.]]


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


## Regresja liniowa

Aby porównać jakość dopasowania i wpływ standardyzacji rozpoczniemy od zwykłej, nieregularyzowanej regresji liniowej.


In [3]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg_s = LinearRegression()

reg.fit(X,y)
reg_s.fit(X, y2)
print(' R^2:', reg.score(X, y),'\n',
      'coef:', reg.coef_,'\n',
     'intercept:', reg.intercept_)

print(' R^2:', reg_s.score(X, y2),'\n',
      'coef:', reg_s.coef_,'\n',
     'intercept:', reg_s.intercept_)


 R^2: 0.5177494254132934 
 coef: [ -10.01219782 -239.81908937  519.83978679  324.39042769 -792.18416163
  476.74583782  101.04457032  177.06417623  751.27932109   67.62538639] 
 intercept: 152.1334841628965
 R^2: 0.5177494254132934 
 coef: [[ -0.13001884  -3.11430123   6.75066232   4.21254835 -10.28733834
    6.1910424    1.31216923   2.29936317   9.75614628   0.87818624]] 
 intercept: [7.12105017e-15]


Widać przeskalowanie współczynników, ale to wciaż jest ten sam model, tylko w innej skali. 


# Regresja grzebietowa

Regresja grzebietowa (ridge regression) zwana również regularyzacją Tichonowa. Posługuje się ona 
funkcjonałem kosztu podstaci 
$$ L(\theta) = MSE(\theta) + \alpha \frac{1}{2} \sum_{i=1}^n \theta^2_i. $$
Oznacza to, że błąd średniokwadratowy jest minimalizowany jednocześnie z łączną sumą kwadratów wszystkich 
parametrów modelu.

Podobnie jak i w przypadku regresji liniowej i logistycznej mamy dostepne techniki bezpośrednie oraz
spadku gradientu. Prezentujemy kod działania obu


In [4]:
from sklearn.linear_model import Ridge

reg2 = Ridge(alpha=1, solver='cholesky')
reg2_s = Ridge(alpha=1, solver='cholesky')
reg2_s2 = Ridge(alpha=2, solver='cholesky')

reg2.fit(X, y)
reg2_s.fit(X, y2)
reg2_s2.fit(X, y2)

print(' R^2:', reg2.score(X, y),'\n',
      'coef:', reg2.coef_,'\n',
     'intercept:', reg2.intercept_)
print(' R^2:', reg2_s.score(X, y2),'\n',
      'coef:', reg2_s.coef_,'\n',
     'intercept:', reg2_s.intercept_)
print(' R^2:', reg2_s2.score(X, y2),'\n',
      'coef:', reg2_s2.coef_,'\n',
     'intercept:', reg2_s2.intercept_)

 R^2: 0.4512313946799056 
 coef: [  29.46574564  -83.15488546  306.35162706  201.62943384    5.90936896
  -29.51592665 -152.04046539  117.31171538  262.94499533  111.878718  ] 
 intercept: 152.13348416289622
 R^2: 0.4512313946799055 
 coef: [[ 0.38264347 -1.079853    3.97829569  2.61836869  0.07673933 -0.38329512
  -1.97440417  1.52341509  3.41461526  1.45286195]] 
 intercept: [3.35956305e-15]
 R^2: 0.3904656114947056 
 coef: [[ 0.43742669 -0.53294968  2.89627691  1.97651967  0.27194349 -0.03570801
  -1.57213839  1.3468778   2.53357596  1.29169214]] 
 intercept: [2.34896853e-15]


Widać pogorszenie współczynnika $R^2$, powiązane ze wzrostem $\alpha$, za to zmniejszenie wartości bezwzględnych współczynników regresji.

Drugi istotny wniosek &mdash; standardyzacja zmiennej objaśnianej nie jest konieczna: wygląda na to, że i tak jest wykonywana niejawnie.


Drugi wariant wykorzystuje spadek gradientu. 


In [5]:
from sklearn.linear_model import SGDRegressor

regressor = SGDRegressor(penalty='l2', loss='squared_error')
regressor.fit(X, y)
y_pred = regressor.predict(X)

from sklearn.metrics import explained_variance_score
explained_variance_score(y, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


0.43678982655165144

In [6]:
from sklearn.linear_model import SGDRegressor

regressor = SGDRegressor(penalty='l2', loss='squared_error')
regressor.fit(X, y2[:,0])
y_pred = regressor.predict(X)

from sklearn.metrics import explained_variance_score
explained_variance_score(y2, y_pred)

0.11487637128141082

Zwraca uwagę i zaskakuje porównanie wyników. Przyczyną jest pewnie słaba stabilność numeryczna przy wielu zmiennych. Istotna też jest ukryta domyślna wartość $\alpha = 0.0001$.


# Regresja LASSO

LASSO jest skrótem od least absolute shrinkache and selection operator. Idea tej regresji jest 
podobna do grzebietowej jednak dołąca ona sumę modułów czyli rozmiar modelu w sensi metryki $l_1$.
Dokładnie minimalizowany jest funkcjonał:
$$ L(\theta) = MSE(\theta) + \alpha \sum_{i=1}^n |\theta_i|. $$

Do ciekawych własności tej regresji zaliczyć można to, że cechy nie mające statystycznie 
istotnego wpływu w regresji są zerowane. Zatem sam algorytm dokonuje niejawnie również selekcji 
zmiennych do modelu. Poniższy przykład (wzięcie większej wartości $\alpha$) pokazuje z kolei, że selekcja cech jest chyba wykonywana zbyt agresywnie. Warto porównać z wersją bez standardyzacji.


In [7]:
from sklearn.linear_model import Lasso

reg3 = Lasso(alpha=0.1)
reg3_s = Lasso(alpha=0.1)
reg3_s2 = Lasso(alpha=0.1/77) # alpha zmniejszona mniej więcej tylokrotnie ile wartości y2
reg3.fit(X, y)
reg3_s.fit(X, y2)
reg3_s2.fit(X, y2)

print(' R^2:', reg3.score(X, y),'\n',
      'coef:', reg3.coef_,'\n',
     'intercept:', reg3.intercept_)
print(' R^2:', reg3_s.score(X, y2),'\n',
      'coef:', reg3_s.coef_,'\n',
     'intercept:', reg3_s.intercept_)
print(' R^2:', reg3_s2.score(X, y2),'\n',
      'coef:', reg3_s2.coef_,'\n',
     'intercept:', reg3_s2.intercept_)

 R^2: 0.5088400794027146 
 coef: [  -0.         -155.36288234  517.18201661  275.08235083  -52.54026923
   -0.         -210.15975349    0.          483.91440913   33.67282148] 
 intercept: 152.13348416289642
 R^2: 0.0 
 coef: [ 0.  0.  0.  0.  0.  0. -0.  0.  0.  0.] 
 intercept: [-1.68794089e-16]
 R^2: 0.5088392220009011 
 coef: [-0.         -2.01747427  6.71613936  3.57219246 -0.6821993  -0.
 -2.72911949  0.          6.28407925  0.43723943] 
 intercept: [6.27653463e-15]


Nieintuicyjnym zaskoczeniem jest dla mnie fakt, że po standardyzacji wyników do sensownego działania wymagana jest tyleż samo mniejsza wartość parametru $\alpha$. Według dokumentacji powinno być wręcz odwrotnie. Najwyraźniej przy przekształconych danych dokonywana jest agresywne zerowanie odpowiednich współcznynników z wykorzystaniem tego parametru.


Drugi wariant wykorzystuje spadek gradientu. Równiez widać  gorszą jakosc wyniku &mdash; ta optymalizacja jest trudna przy bardzo wielu zmiennych.


In [9]:
from sklearn.linear_model import SGDRegressor

regressor = SGDRegressor(penalty='l1', loss='squared_error', alpha = 0.1)
regressor.fit(X, y)
y_pred = regressor.predict(X)

from sklearn.metrics import explained_variance_score
explained_variance_score(y, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


0.42751201917265924

In [10]:
from sklearn.linear_model import SGDRegressor

regressor = SGDRegressor(penalty='l1', loss='squared_error')
regressor.fit(X, y2[:,0])
y_pred = regressor.predict(X)

from sklearn.metrics import explained_variance_score
explained_variance_score(y2, y_pred)

0.11274817466921927


# Metoda elastycznej siatki 

Regresja z użyciem kombinacji obu wcześniejszych regularyzacji nazywa się techniką elastycznej siatki.
Minimalizuje ona funkcjonał postaci
$$ L(\theta) = MSE(\theta) + r \alpha \sum_{i=1}^n |\theta_i| + \frac{1-r}{2} \alpha \sum_{i=1}^n \theta_i^2. $$


In [11]:
from sklearn.linear_model import ElasticNet

reg4 = ElasticNet(alpha = 0.1, l1_ratio=0.5)
reg4_s = ElasticNet(alpha = 0.1/77, l1_ratio=0.5)
reg4.fit(X, y)
reg4_s.fit(X, y2)

print(' R^2:', reg4.score(X, y),'\n',
      'coef:', reg4.coef_,'\n',
     'intercept:', reg4.intercept_)
print(' R^2:', reg4_s.score(X, y2),'\n',
      'coef:', reg4_s.coef_,'\n',
     'intercept:', reg4_s.intercept_)

 R^2: 0.10298926753450965 
 coef: [ 10.28632698   0.28597648  37.46464319  27.5448988   11.1088504
   8.35588419 -24.12080817  25.50548821  35.46575709  22.89498054] 
 intercept: 152.13348416289597
 R^2: 0.49661168769208175 
 coef: [ 0.         -1.75584275  5.53757111  3.34992039 -0.08274315 -0.68762414
 -2.42103034  1.13729957  4.75750414  1.19534065] 
 intercept: [4.84155793e-15]


W powyższym przykładzie widać przewagę standardyzacji &mdash; dużo łatwiej dobrać rozsądną wartość stosunku wag dwóch metod względem siebie (parametru $r$ powyżej).

# Co to daje?

Poniżej pokażemy, że wskazane metody (nawet jeśli nie w pełni jasne w stosowaniu, co wskazano powyżej) faktycznie mog pomóc w sytuacji ryzyka przeuczenia. Zasymulujemy to ryzyko dzieląc dane na (małą) część uczącą i dużą testową. Żeby była powtarzalność nie skorzystamy z gotowej funkcji, a wezmiemy pierwszych 40 obserwacji do nauki. Mimo powyższych trudności skorzystamy ze standaryzowanej kolumny *target*.

In [12]:
#from sklearn.model_selection import train_test_split
#X_train, X_test , y_train, y_test = train_test_split(X, y2, train_size = 40)

X_train = X[0:40]
X_test = X[40:]
y_train = y2[0:40,0]
y_test = y2[40:,0]
X_train.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


###  Regresja liniowa
Poniżej widać pewne przeuczenie modelu regresji liniowej.


In [13]:
reg5 = LinearRegression()

reg5.fit(X_train,y_train)
print(' R^2:', reg5.score(X_train, y_train),'\n',
      'coef:', reg5.coef_,'\n',
     'intercept:', reg5.intercept_, '\n',
      ' R^2 - testowy', reg5.score(X_test, y_test))

# dla porównania zwykła regresja liniowa
print(' R^2:', reg_s.score(X, y2),'\n',
      'coef:', reg_s.coef_,'\n',
     'intercept:', reg_s.intercept_)


 R^2: 0.6897762205586805 
 coef: [ -1.97678013  -1.8165416    5.13057043   4.6763912  -17.70405487
   4.80952831   7.88866699  10.16728776  18.78243966  -5.64894025] 
 intercept: -0.08660621048851444 
  R^2 - testowy 0.28795059767697684
 R^2: 0.5177494254132934 
 coef: [[ -0.13001884  -3.11430123   6.75066232   4.21254835 -10.28733834
    6.1910424    1.31216923   2.29936317   9.75614628   0.87818624]] 
 intercept: [7.12105017e-15]


### Po zastosowaniu regularyzacji

Poniżej obliczenia przy stosowaniu regularyzacji. Widać wyraźną poprawę dopasowania na zbiorze testowym względem zwykłej regresji liniowej. Do ustalenia pozostaje jeszcze sposób doboru parametru $\alpha$ &mdash; jest to trudne przy małym rozmiarze zbioru danych, a głównie w takiej sytuacji używa się tych metod. Najlepszym rozwiązaniem wydaje się walidacja krzyżowa. Na obecnym etapie zostanie to pominięte. 

In [14]:
reg6 = Ridge(alpha=0.2, solver='cholesky')
reg7 = Lasso(alpha=0.2/77)

reg6.fit(X_train, y_train)
reg7.fit(X_train, y_train)

print(' R^2:', reg6.score(X_train, y_train),'\n',
      'coef:', reg6.coef_,'\n',
     'intercept:', reg6.intercept_, '\n',
      ' R^2 - testowy', reg6.score(X_test, y_test))
print('------------------------------------')
print(' R^2:', reg7.score(X_train, y_train),'\n',
      'coef:', reg7.coef_,'\n',
     'intercept:', reg7.intercept_, '\n',
      ' R^2 - testowy', reg7.score(X_test, y_test))


 R^2: 0.390837690153234 
 coef: [-3.56756496e-02 -1.93409855e-01  2.19550930e+00  1.46590424e+00
 -1.22034597e-03 -9.19583368e-01 -1.18772282e+00  1.37910088e+00
  3.58943031e+00  8.93813429e-01] 
 intercept: 0.015314593244775805 
  R^2 - testowy 0.36362197426060383
------------------------------------
 R^2: 0.616343904121351 
 coef: [-0.75590404 -0.62740668  3.89662161  2.03883486 -3.85563868 -0.
 -0.          1.84494915 12.66213507 -1.37486128] 
 intercept: -0.012523740261601128 
  R^2 - testowy 0.42860202968264605
